In [3]:
import tensorflow as tf

In [4]:
import pandas as pd

In [5]:
from pandas import read_csv

In [6]:
columns=['crim','zn','indus','chas','nox','rm','age','dis','rad','tax','ptratio','b','lstat','medv']
path="https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
d_frame=read_csv(path, sep='\s+', names=columns)
print(d_frame.head(),d_frame.shape,sep="\n")

      crim    zn  indus  chas    nox     rm   age     dis  rad    tax  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296.0   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242.0   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242.0   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222.0   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222.0   

   ptratio       b  lstat  medv  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  
(506, 14)


In [7]:
training_set =d_frame.iloc[0:350,:]
test_set = d_frame.iloc[350:450,:]
prediction_set = d_frame.iloc[450:507,:]


# using pandas estimator

In [6]:
print(training_set.shape, test_set.shape, prediction_set.shape)

(350, 14) (100, 14) (56, 14)


In [8]:
from sklearn import datasets
import itertools

In [31]:
FEATURES = ['crim','zn','indus','chas','nox','rm','age','dis','rad','tax','ptratio','b','lstat']
LABEL = "medv"


In [32]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [28]:
feature_cols[0]

NumericColumn(key='crim', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [18]:
estimator = tf.estimator.LinearRegressor(    
        feature_columns=feature_cols,   
        model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000211D83194C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Tensorflow will automatically create a file named train in your working directory. You need to use this path to access the Tensorboard.

In [39]:
def get_input_fn(data_set, num_epochs=None, n_batch = 128, shuffle=True):    
         return tf.compat.v1.estimator.inputs.pandas_input_fn(       
         x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),   #creating dataframe from dictionary{column_name:column_data}    
         y = pd.Series(data_set[LABEL].values),       
         batch_size=n_batch,          
         num_epochs=num_epochs,       
         shuffle=shuffle)

In [40]:
get_input_fn(training_set, num_epochs=None, n_batch = 128, shuffle=False)

<function tensorflow_estimator.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn()>

In [43]:
# train the estimator(estimator is your model, check above)
estimator.train(input_fn=get_input_fn(training_set, num_epochs=None, n_batch = 128, shuffle = False),steps=1000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 200 into train\model.ckpt.
INFO:tensorflow:loss = 28.24475, step = 200
INFO:tensorflow:global_step/sec: 59.6604
INFO:tensorflow:loss = 49.88719, step = 300 (1.810 sec)
I

In [44]:
#evaluate the fit of your model on the test set
ev = estimator.evaluate(    
          input_fn=get_input_fn(test_set,                          
          num_epochs=1,                          
          n_batch = 128,                          
          shuffle=False))


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-11T18:28:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train\model.ckpt-1200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-11-18:28:45
INFO:tensorflow:Saving dict for global step 1200: average_loss = 740.91156, global_step = 1200, label/mean = 15.955001, loss = 740.9

In [45]:
#You can print the loss with the code
loss_score = ev["loss"]
print("Loss: {}".format(loss_score))

Loss: 740.9115600585938


In [46]:
training_set['medv'].describe()

count    350.000000
mean      25.068857
std        8.473240
min       11.800000
25%       19.500000
50%       22.900000
75%       28.700000
max       50.000000
Name: medv, dtype: float64

From the summary statistic above, you know that the average price for a house is 25 thousand, with a minimum price of 11 thousands and maximum of 50 thousand. The model makes a typical error of 0.7k dollars.

In [49]:
# use the estimator predict to estimate the value
y=estimator.predict(input_fn=get_input_fn(prediction_set,
                                       num_epochs=1,
                                       n_batch=128,
                                       shuffle=False))


In [50]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train\model.ckpt-1200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([9.708919], dtype=float32), array([33.164738], dtype=float32), array([34.512505], dtype=float32), array([39.050373], dtype=float32), array([12.009208], dtype=float32), array([9.916839], dtype=float32)]


# using numpy estimator
how to train the model using a numpy estimator to feed the data. The method is the same exept that you will use numpy_input_fn estimator.

In [9]:
def prepare(dframe):
    x_values=dframe.iloc[:,:-1].values
    y_values=dframe.iloc[:,-1].values
    return x_values, y_values

In [10]:
X_train, y_train = prepare(training_set)
X_test, y_test = prepare(test_set)

In [11]:
X_predict,y_predict = prepare(prediction_set)

In [12]:
print(X_train.shape, X_test.shape, X_predict.shape)

(350, 13) (100, 13) (56, 13)


In [70]:
# construct the feature columns
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train.shape[1:])] # where x=pd.DataFrame({k: d_frame[k].values for k in FEATURES})
# feature_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [71]:
#The estimator is defined as before, you instruct the feature columns and where to save the graph.
estimator = tf.estimator.LinearRegressor(    
         feature_columns=feature_columns,    
         model_dir="train1")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001DAE884C548>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [76]:
len(y_train)

350

In [73]:
{"x": X_train} # it is ndarray of rows of x_train

{'x': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [1.8700e-02, 8.5000e+01, 4.1500e+00, ..., 1.7900e+01, 3.9243e+02,
         6.3600e+00],
        [1.5010e-02, 8.0000e+01, 2.0100e+00, ..., 1.7000e+01, 3.9094e+02,
         5.9900e+00],
        [2.8990e-02, 4.0000e+01, 1.2500e+00, ..., 1.9700e+01, 3.8985e+02,
         5.8900e+00]])}

In [74]:
# create input to Train the estimator
train_input=tf.compat.v1.estimator.inputs.numpy_input_fn(
                    x={"x": X_train},
                    y=y_train,
                    batch_size=128,
                    shuffle=False,
                    num_epochs=None)

In [75]:
# train estimator
estimator.train(input_fn=train_input,steps=5000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train1\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10000 into train1\model.ckpt.
INFO:tensorflow:loss = 11.273365, step = 10000
INFO:tensorflow:global_step/sec: 217.678
INFO:tensorflow:loss = 20.15383, step = 10100 (0

In [19]:
# create input to evaluate the model
eval_input=tf.compat.v1.estimator.inputs.numpy_input_fn(
                    x={"x":X_test},
                    y=y_test,
                    batch_size=128,
                    shuffle=False,
                    num_epochs=1)

In [80]:
# evaluate the model
estimator.evaluate(input_fn=eval_input, steps=None)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-11T19:29:43Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train1\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-11-19:29:45
INFO:tensorflow:Saving dict for global step 5000: average_loss = 1911.6772, global_step = 5000, label/mean = 15.955001, loss = 1911

{'average_loss': 1911.6772,
 'label/mean': 15.955001,
 'loss': 1911.6772,
 'prediction/mean': 46.93169,
 'global_step': 5000}

In [81]:
# to create input to test model
predict_input=tf.compat.v1.estimator.inputs.numpy_input_fn(
                    x={"x":X_predict},
                    batch_size=128,
                    shuffle=False,
                    num_epochs=1)

In [83]:
y = estimator.predict(predict_input) 

In [84]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train1\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([18.301329], dtype=float32), array([36.191578], dtype=float32), array([36.564926], dtype=float32), array([45.308662], dtype=float32), array([23.101326], dtype=float32), array([16.082914], dtype=float32)]
